In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
my_result = data_test.groupby('user_id')['item_id'].unique().reset_index()
my_result.columns=['user_id', 'actual']
my_result['actual'] = my_result['actual'].apply(lambda x: list(x))

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights.iloc[:,0])
    probability = np.array(items_weights.iloc[:,1])
    recs = np.random.choice(items, size=n, replace=True, p=probability)
    
    return recs.tolist()

Сделайте предсказания

In [9]:
sales_log = data_train.groupby('item_id')['sales_value'].sum().reset_index()    

sales_log['sales_value'] = np.log(sales_log['sales_value'], out=np.zeros_like(sales_log['sales_value']), where=(sales_log['sales_value'] !=0))

sales_log.loc[sales_log['sales_value'] < 0, 'sales_value'] = 0

sales_log['sales_value'] = sales_log['sales_value']/(sales_log['sales_value'].sum())

my_result['weighted_random_recs_1'] = my_result['user_id'].apply(lambda x: weighted_random_recommendation(sales_log, n=5))

In [10]:
sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
sales['sales_value'] = sales['sales_value']/(sales['sales_value'].sum())

my_result['weighted_random_recs_2'] = my_result['user_id'].apply(lambda x: weighted_random_recommendation(sales, n=5))

In [11]:
sales_num = data_train.groupby('item_id')['quantity'].sum().reset_index()
sales_num['quantity'] = sales_num['quantity']/(sales_num['quantity'].sum())

my_result['weighted_random_recs_3'] = my_result['user_id'].apply(lambda x: weighted_random_recommendation(sales_num, n=5))

In [12]:
my_result.head(5)

,user_id,actual,weighted_random_recs_1,weighted_random_recs_2,weighted_random_recs_3
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6703878, 7166638, 5569419, 1045437, 888167]","[865233, 10149265, 9242471, 896613, 5996054]","[6533889, 6534178, 6534178, 6534178, 6534178]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[930870, 905436, 12355478, 955614, 1086304]","[6534330, 6534178, 873887, 8065410, 920654]","[6534178, 6534178, 6534178, 6534178, 6534178]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1128206, 8122102, 12672883, 1021709, 1078127]","[947858, 15926844, 6533765, 1076875, 6424251]","[6534178, 6534178, 6534178, 6534178, 6534178]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[12427374, 843111, 986912, 953713, 876695]","[1094955, 1048507, 1048962, 892573, 981494]","[6534178, 6533889, 6534178, 6534166, 6534178]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[12604786, 1107066, 828935, 8204459, 5727795]","[839162, 13876458, 13115938, 1096317, 8090541]","[6534178, 6534178, 6533889, 6534178, 6534178]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [13]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[1019719, 2332987, 5550112, 9858801, 1072575]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[839002, 13158305, 12354593, 8199063, 15597006]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [14]:
type(result['actual'][0])

str

In [20]:
result['weighted_random_recs_1'] = my_result['weighted_random_recs_1']
result['weighted_random_recs_2'] = my_result['weighted_random_recs_2']
result['weighted_random_recs_3'] = my_result['weighted_random_recs_3']

result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recs_1,weighted_random_recs_2,weighted_random_recs_3
0,1,[ 821867 834484 856942 865456 889248 ...,"[19719, 2332987, 5550112, 9858801, 1072575]","[534178, 6533889, 1029743, 6534166, 1082185]","[99999, 1082185, 981760, 1127831, 995242]","[82185, 999999, 981760, 1127831, 1098066]","[82185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[6703878, 7166638, 5569419, 1045437, 888167]","[865233, 10149265, 9242471, 896613, 5996054]","[6533889, 6534178, 6534178, 6534178, 6534178]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[39002, 13158305, 12354593, 8199063, 15597006]","[534178, 6533889, 1029743, 6534166, 1082185]","[99999, 1082185, 981760, 1098066, 995242]","[82185, 1098066, 981760, 999999, 826249]","[82185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[930870, 905436, 12355478, 955614, 1086304]","[6534330, 6534178, 873887, 8065410, 920654]","[6534178, 6534178, 6534178, 6534178, 6534178]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[4112110, 13671679, 9651712, 13416730, 884319]","[534178, 6533889, 1029743, 6534166, 1082185]","[99999, 1082185, 981760, 1127831, 995242]","[82185, 999999, 981760, 1127831, 1098066]","[82185, 981760, 1127831, 999999, 878996]","[1082185, 1029743, 6534178, 1127831, 995785]","[1128206, 8122102, 12672883, 1021709, 1078127]","[947858, 15926844, 6533765, 1076875, 6424251]","[6534178, 6534178, 6534178, 6534178, 6534178]"


In [25]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [27]:
alg = list(result.columns)[2:] 
alg.remove('own_purchases')
for i in alg:
    precision = result.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1).mean()
    print(i, precision)

random_recommendation 0.0
popular_recommendation 0.0
itemitem 0.0
cosine 0.0
tfidf 0.0
weighted_random_recs_1 0.0
weighted_random_recs_2 0.0
weighted_random_recs_3 0.0


In [28]:
print(precision_at_k(result['own_purchases'],result['actual'] ))

0.0
